<a href="https://colab.research.google.com/github/GatienVilain/Toxic-Comment-classification/blob/Etude-du-jeu-de-donnees/Classification_commentaires_toxiques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des packages

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [81]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [82]:
data_train = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')
data_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [83]:
data_test = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test.csv')
data_test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [84]:
data_test_labels = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test_labels.csv')
data_test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


# Etude du jeu de données

## a. Train Data

On sépare les commentaires des catégories

In [95]:
x_train = data['comment_text']
x_train[59]

"Why can't you believe how fat Artie is? Did you see him on his recent appearence on the Tonight Show with Jay Leno? He looks absolutely AWFUL! If I had to put money on it, I'd say that Artie Lange is a can't miss candidate for the 2007 Dead pool!   \n\n  \nKindly keep your malicious fingers off of my above comment, . Everytime you remove it, I will repost it!!!"

In [87]:
x_train[50:60]

50    "\n\nBI, you said you wanted to talk\n\nAt the...
51    GET FUCKED UP. GET FUCKEEED UP.  GOT A DRINK T...
52    Are you threatening me for disputing neutralit...
53    Thanks! Undeletion was more than I'd hoped for...
54    Awesome! Then I'll simply disregard your notic...
55    Stupid peace of shit stop deleting my stuff as...
56    =Tony Sidaway is obviously a fistfuckee. He lo...
57    "\n\n GA Review II \n\nI'm sorry to say this, ...
58    My Band Page's deletion. You thought I was gon...
59    Why can't you believe how fat Artie is? Did yo...
Name: comment_text, dtype: object

On remarque que la colonne id n’est pas utile pour notre entrainement

In [88]:
y_train = data.drop(columns=['id', 'comment_text'])
y_train[50:60]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
50,0,0,0,0,0,0
51,1,0,1,0,0,0
52,0,0,0,0,0,0
53,0,0,0,0,0,0
54,0,0,0,0,0,0
55,1,1,1,0,1,0
56,1,0,1,0,1,0
57,0,0,0,0,0,0
58,1,0,1,0,0,0
59,1,0,0,0,0,0


In [97]:
number_of_lines = y_train['toxic'].count()
print('Nombre de donnée : ', number_of_lines)

Nombre de donnée :  159571


In [109]:
print("Pourcentages des différentes catégories parmi l'ensemble de données")
y_train.sum()/number_of_lines*100

Pourcentages des différentes catégories parmi l'ensemble de données


toxic            9.584448
severe_toxic     0.999555
obscene          5.294822
threat           0.299553
insult           4.936361
identity_hate    0.880486
dtype: float64

## b. Test Data

In [111]:
# Your Code
x_test = data['comment_text']
x_test[59]

"Why can't you believe how fat Artie is? Did you see him on his recent appearence on the Tonight Show with Jay Leno? He looks absolutely AWFUL! If I had to put money on it, I'd say that Artie Lange is a can't miss candidate for the 2007 Dead pool!   \n\n  \nKindly keep your malicious fingers off of my above comment, . Everytime you remove it, I will repost it!!!"

In [112]:
x_test[50:60]

50    "\n\nBI, you said you wanted to talk\n\nAt the...
51    GET FUCKED UP. GET FUCKEEED UP.  GOT A DRINK T...
52    Are you threatening me for disputing neutralit...
53    Thanks! Undeletion was more than I'd hoped for...
54    Awesome! Then I'll simply disregard your notic...
55    Stupid peace of shit stop deleting my stuff as...
56    =Tony Sidaway is obviously a fistfuckee. He lo...
57    "\n\n GA Review II \n\nI'm sorry to say this, ...
58    My Band Page's deletion. You thought I was gon...
59    Why can't you believe how fat Artie is? Did yo...
Name: comment_text, dtype: object

# Préparation des données

In [ ]:
# Your Code

# Entraînement du modèle baseline

In [ ]:
# Your Code

# Itération de la modélisation 

In [ ]:
# Your Code 